<a href="https://colab.research.google.com/github/estorl03-tech/AI-AGENT/blob/main/AI%E3%82%A8%E3%83%BC%E3%82%B8%E3%82%A7%E3%83%B3%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langgraph langchain_openai langchain_core tweepy requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 10.5 MB/s eta 0:00:00


## 応用AIエージェント

- ニュースを取得
- 文書を要約
- Tweetを投稿する


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from google.colab import userdata

import gradio as gr
import requests
import tweepy

## APIキー
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
NEWS_API_KEY = userdata.get('NEWS_API_KEY')
BEARER_TOKEN = userdata.get('BEARER_TOKEN')
X_API_KEY = userdata.get('X_API_KEY')
X_API_KEY_SECRET = userdata.get('X_API_KEY_SECRET')
ACCESS_TOKEN = userdata.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = userdata.get('ACCESS_TOKEN_SECRET')

# 最新のニュースをひとつ取得するツール
@tool
def get_news(query: str) -> str:
  """キーワードに合致する最新ニュースを一件取得してタイトルと説明を返す"""
  url = "http://newsapi.org/v2/everything"
  params = {
      "q": query,
      "sortBy": "publishedAt",
      "pageSize": 1,
      "apiKey": NEWS_API_KEY
  }

  article = requests.get(url, params).json()["articles"][0]
  return f"{article['title']} - {article['description']}"

# 140字以内で要約するツール
@tool
def summarize(text: str) -> str:
  """テキストを要約して返す"""
  llm = ChatOpenAI(model = "gpt-4o-mini-2024-07-18", temperature = 0.1, openai_api_key = OPENAI_API_KEY)
  return llm.invoke([SystemMessage(content="あなたは優秀な要約家です。以下のテキストを140字以内で簡潔に要約してください。）"), HumanMessage(content = text)]).content

# ツイートをするツール
@tool
def post_tweet(text: str) -> str:
  """テキストをツイートする"""
  client = tweepy.Client(
    bearer_token = BEARER_TOKEN,
    consumer_key = X_API_KEY,
    consumer_secret = X_API_KEY_SECRET,
    access_token = ACCESS_TOKEN,
    access_token_secret = ACCESS_TOKEN_SECRET
  )

  tweet = client.create_tweet(text = text)
  return f"{tweet.data['id']}"

# AIエージェント作成
def run_agent(user_input: str) -> str:
  tools = [get_news, summarize, post_tweet]

  system_prompt = "ユーザーの入力に対して、get_newsとsummarizeとpost_tweetから適切なツールを使って答えてください"

  llm = ChatOpenAI(model = "gpt-4o-mini-2024-07-18", temperature = 0, openai_api_key = OPENAI_API_KEY)
  agent = create_react_agent(llm, tools, prompt=system_prompt)

  response =  agent.invoke({"messages": [HumanMessage(content = user_input)]})
  print(response["messages"][-1].content)

  return response

# run_agentをそのまま使う
def agent_interface(user_input):
    try:
        response = run_agent(user_input)
        # LangGraphは messages を返すので、最後のAIMessage.contentを取り出す
        messages = response["messages"]
        last_message = messages[-1]

        # AIMessageの本文
        if hasattr(last_message, "content"):
          output_text = last_message.content
        else:
          output_text = str(last_message)

        return f"✅ 完了しました！\n\n{output_text}"

    except Exception as e:
        return f"エラー: {e}"

with gr.Blocks() as demo:
    gr.Markdown("## ニュース取得 → 要約 → ツイート AIエージェント Demo")

    with gr.Row():
        with gr.Column(scale=3):
            user_input = gr.Textbox(
                label="入力（例: AI, 経済, スポーツ）",
                placeholder="気になるトピックを入力してください"
            )
            run_button = gr.Button("実行")

        with gr.Column(scale=4):
            output = gr.Textbox(
                label="出力",
                lines=5
            )

    run_button.click(fn=agent_interface, inputs=user_input, outputs=output)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3ccc8e7778a1a0937.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
user_input = "人工知能に関するニュースを要約してツイートして"
response = run_agent(user_input)
response

最新の人工知能に関するニュースをツイートしました。内容は以下の通りです：

「米巨大テック企業、AI人材争奪戦で自らを脅かす」
AI人材への需要が高まる中、企業は新しい技術の進展や倫理的課題、規制の動向に対応しなければならない。


{'messages': [HumanMessage(content='人工知能に関するニュースを要約してツイートして', additional_kwargs={}, response_metadata={}, id='9f0e09b5-bded-443a-ae44-d5db883d3ed8'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0eIcE9En1f1fDn654kcOjr1A', 'function': {'arguments': '{"query": "人工知能"}', 'name': 'get_news'}, 'type': 'function'}, {'id': 'call_ug9Spw6nnR6vvad5h0kNhmjU', 'function': {'arguments': '{"text": "人工知能に関する最新のニュース"}', 'name': 'summarize'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 150, 'total_tokens': 205, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6kfVWbjMA0CS5NBLVtYAF8ifRFkt', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs'